# fNIRS signal Classification using Deep learning
---
## Functional Near-Infrared Spectroscopy (fNIR or fNIRS), is the use of near-infrared spectroscopy (NIRS) for the purpose of functional neuroimaging. Using fNIR, brain activity is measured through hemodynamic responses associated with neuron behaviour. fNIRS signals have been successfully implemented as a control signal for BCI systems.Keeping that in mind this project intends to improve the detection accuracy of such BCI systems using Deep Neural Networks.
### The problem that this project aims to tackle is to diffrentiate between a rest and active state of a human using fNIRS signals of the person in question.

# Loading and preprocessing of data 
---
## We shall first begin by loading the datasets provided. The dataset has been filtered and preprocessed before hand making further classifications easier. It is composed into two parts Oxy and Deoxy. This refers to the different blood channels that were used to extract the fNIRS signals. We will standardize the data making its Mean 0 and its Variance 1.

In [0]:
# import library and data
import numpy as np
import scipy.io as sio
from sklearn import preprocessing

fNIRS_data_oxy = sio.loadmat('data_oxy.mat')
# preproccesing data for zero mean and 1 variance
data_oxy = preprocessing.scale(np.array(fNIRS_data_oxy['data']))
labels_oxy = np.array(fNIRS_data_oxy['labels'])

fNIRS_data_deoxy = sio.loadmat('data_deoxy.mat')
data_deoxy = preprocessing.scale(np.array(fNIRS_data_deoxy['data']))
labels_deoxy = np.array(fNIRS_data_deoxy['labels'])

data_hybrid = np.concatenate((data_oxy, data_deoxy),axis = 1)
labels_hybrid = labels_oxy

# Defining Deep Neural Network Models
---
## For the task at hand 3 different models will be created firstly for the Oxy dataset secondly for the Deoxy dataset and thirdly a combination of the two. 
## For some experimentation we also added a graph model that takes data from both the Oxy and Deoxy datasets passes them through the some layers and then tries to classify using the new embeddings made during the forward pass.   

In [0]:
from keras.layers import Input
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization

def oxymdl():
    model = Sequential()
    model.add(Dense(100, input_dim=16, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(100, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(100, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(100, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(100, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(100, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(50, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(10, activation='relu'))
    model.add(Dropout(.3))
    model.add(Dense(1, activation = 'sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def deoxymdl():
    model = Sequential()
    model.add(Dense(100, input_dim=16, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(100, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(100, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(100, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(100, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(100, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(50, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(10, activation='relu'))
    model.add(Dropout(.3))
    model.add(Dense(1, activation = 'sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
    
def hbrdmdl():
    model = Sequential()
    model.add(Dense(100, input_dim=16, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(100, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(100, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(100, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(100, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(100, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(50, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(10, activation='relu'))
    model.add(Dropout(.3))
    model.add(Dense(1, activation = 'sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def hbrdgrfmdl():
    in_ox = Input(shape=(16,))
    in_deox = Input(shape=(16,))
    
    ox = Dense(100, activation='relu')(in_ox)
    ox = BatchNormalization()(ox)
    deox = Dense(100, activation='relu')(in_deox)
    deox = BatchNormalization()(deox)
    ox = Dense(100, activation='relu')(ox)
    ox = BatchNormalization()(ox)
    deox = Dense(100, activation='relu')(deox)
    deox = BatchNormalization()(deox)
    ox = Dense(100, activation='relu')(ox)
    ox = BatchNormalization()(ox)
    deox = Dense(100, activation='relu')(deox)
    deox = BatchNormalization()(deox)
    ox = Dense(100, activation='relu')(ox)
    ox = BatchNormalization()(ox)
    deox = Dense(100, activation='relu')(deox)
    deox = BatchNormalization()(deox)
    ox = Dense(100, activation='relu')(ox)
    ox = BatchNormalization()(ox)
    deox = Dense(100, activation='relu')(deox)
    deox = BatchNormalization()(deox)
    x = concatenate([ox, deox])
    x = Dense(50, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(10, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(.3)(x)
    out = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=[in_ox, in_deox], outputs=[out])

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

Using Theano backend.


# Oxy model
---
## Here we check the accuracy of the Oxy model using cross validation. 5 folds are created and the model is trained and tested upon them
## The mean accuracy for the Oxy model is 99.40%

In [0]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import numpy as np

np.random.seed(7)

model = KerasClassifier(build_fn=oxymdl, epochs=20, batch_size=100, verbose=0)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
results = cross_val_score(model, data_oxy, labels_oxy, cv=kfold)
print(results)
print(results.mean())

[ 0.99193548  0.99193548  0.99731183  0.99641577  0.99282511]
0.994084735683


# Deoxy model
---
## Here we check the accuracy of the Deoxy model using cross validation. 5 folds are created and the model is trained and tested upon them
## The mean accuracy for the Deoxy model is 99.39%

In [0]:
model = KerasClassifier(build_fn=deoxymdl, epochs=20, batch_size=100, verbose=0)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
results = cross_val_score(model, data_deoxy, labels_deoxy, cv=kfold)
print(results)
print(results.mean())

[ 0.99193548  0.99193548  0.99731183  0.99551971  0.99282511]
0.993905524214


# Hybrid model
---
## Here we check the accuracy of the Hybrid model using cross validation. 5 folds are created and the model is trained and tested upon them
## The mean accuracy for the Hybrid model is 99.44%

In [0]:
model = KerasClassifier(build_fn=hbrdmdl, epochs=20, batch_size=100, verbose=0)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
results = cross_val_score(model, data_hybrid, labels_hybrid, cv=kfold)
print(results)
print(results.mean())

[ 0.99283154  0.99193548  0.99731183  0.99641577  0.99372197]
0.99444331935


# Hybrid Graph model
---
## Here we train and check the accuracy of the Hybrid Graph model. The model is trained upon 85% and tested on 15% of the dataset
## The test accuracy for the Oxy model is 99.28%

In [0]:
from sklearn.model_selection import train_test_split
graph_model = hbrdgrfmdl()
X1_train, X1_test, Y1_train, Y1_test = train_test_split(data_oxy, labels_oxy, random_state = 42, test_size = .15)
X2_train, X2_test, Y2_train, Y2_test = train_test_split(data_deoxy, labels_deoxy, random_state = 42, test_size = .15)

graph_model.fit([X1_train, X2_train], [Y1_train], epochs=20, batch_size=100, validation_split = .15, verbose = 1)
scores = graph_model.evaluate([X1_test, X2_test], [Y1_test])
print("\n%s: %.2f%%" % (graph_model.metrics_names[1], scores[1]*100))

Train on 4030 samples, validate on 712 samples
Epoch 1/20
4030/4030 [==============================] - 2s 542us/step - loss: 0.2034 - acc: 0.9323 - val_loss: 0.4289 - val_acc: 0.9719
Epoch 2/20
4030/4030 [==============================] - 2s 407us/step - loss: 0.0723 - acc: 0.9886 - val_loss: 0.1934 - val_acc: 0.9972
Epoch 3/20
4030/4030 [==============================] - 2s 397us/step - loss: 0.0568 - acc: 0.9918 - val_loss: 0.0844 - val_acc: 0.9944
Epoch 4/20
4030/4030 [==============================] - 2s 430us/step - loss: 0.0479 - acc: 0.9918 - val_loss: 0.0409 - val_acc: 0.9972
Epoch 5/20
4030/4030 [==============================] - 2s 411us/step - loss: 0.0511 - acc: 0.9893 - val_loss: 0.0306 - val_acc: 0.9972
Epoch 6/20
4030/4030 [==============================] - 2s 412us/step - loss: 0.0449 - acc: 0.9928 - val_loss: 0.0232 - val_acc: 0.9972
Epoch 7/20
4030/4030 [==============================] - 2s 413us/step - loss: 0.0409 - acc: 0.9933 - val_loss: 0.0208 - val_acc: 0.9972
E